In [ ]:
import pickle
from pathlib import Path
from easydict import EasyDict
import pandas as pd
from matplotlib import pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
cfgs, btests = [], []
for p in sorted(Path("optimization").glob("*.pickle")):
    cfg, btest = pickle.load(open(p, "rb"))
    cfgs.append(cfg)
    btests.append(btest)
    # print(p)


In [ ]:
opt_summary = {k:[] for k in cfgs[0].keys()}
for k in opt_summary.keys():
    for cfg in cfgs:
        v = cfg[k]
        if type(v) is EasyDict and "func" in v.keys():
            opt_summary[k].append(str(v.func.name))
        else:
            opt_summary[k].append(v)
            
opt_summary["btest"], opt_summary["ndeals"] = [], []
for btest in btests:
    opt_summary["btest"].append(btest.profits.sum())
    opt_summary["ndeals"].append(len(btest.profits))
    
opt_summary = pd.DataFrame(opt_summary)
opt_summary.sort_values(by=["btest"], ascending=False, inplace=True)
opt_summary = opt_summary[opt_summary.ticker != "NVTK"]
# opt_summary.head(10)

In [ ]:
opt_res_ids = [0, 1, 2]
tickers2plot = ["BTCUSDT", "ETHUSDT"]

on_one_plot = False
plt.figure(figsize=(10, 5*(1 + (1 - on_one_plot)*len(opt_res_ids))))
plt.tight_layout()
for i, opt_res_id in enumerate(opt_res_ids):
    if not on_one_plot:
        plt.subplot(len(opt_res_ids), 1, i+1)
    legend = []
    for ticker2plot in tickers2plot:
        btest_temp = opt_summary[opt_summary.ticker == ticker2plot]
        test_id = btest_temp.index[opt_res_id]
        line = btests[test_id].profits.cumsum()
        plt.plot([pos.close_date for pos in btests[test_id].positions], line, linewidth=2)
        legend.append(f"{test_id} {cfgs[test_id].ticker}-{cfgs[test_id].period}: {cfgs[test_id].body_classifier.func.name}, sl={cfgs[test_id].stops_processor.func.name}, sl-rate={cfgs[test_id].trailing_stop_rate} -> {line[-1]:.0f}")
    plt.legend(legend)
    plt.grid("on")

In [ ]:
opt_res = {"param_set":[], "ticker":[], "btest":[], "ndeals":[], "test_ids":[]}
for i in range(opt_summary.shape[0]):
    exphash, test_ids = "", ""
    for col in opt_summary.columns:
        if col not in ["ticker", "btest", "ndeals", "no_trading_days"]:
            exphash += str(opt_summary[col].iloc[i]) + " "
    opt_res["test_ids"].append(f".{opt_summary.index[i]}")
    opt_res["param_set"].append(exphash)
    opt_res["ticker"].append(f".{opt_summary.ticker.iloc[i]}")
    opt_res["ndeals"].append(opt_summary.ndeals.iloc[i])
    opt_res["btest"].append(opt_summary.btest.iloc[i])

opt_res = pd.DataFrame(opt_res)
opt_res = opt_res.groupby(by="param_set").sum()
opt_res.sort_values(by=["btest"], ascending=False, inplace=True)
# opt_res.head(20)

In [ ]:
plt.figure(figsize=(15, 10))
opt_res_id = 0
legend = []
for test_id in map(int, opt_res.test_ids.iloc[opt_res_id].split(".")[1:]):
    line = btests[test_id].profits.cumsum()
    plt.plot([pos.close_date for pos in btests[test_id].positions], 
             line, 
             linewidth=2)
    legend.append(f"{test_id} {cfgs[test_id].ticker}-{cfgs[test_id].period}: {cfgs[test_id].body_classifier.func.name}, sl={cfgs[test_id].stops_processor.func.name}, sl-rate={cfgs[test_id].trailing_stop_rate} -> {line[-1]:.0f}")
plt.legend(legend)
plt.grid("on")

In [ ]:
import numpy as np
import pandas as pd


def calc_metrics(ts):
    ymax = ts[0]
    twait = 0
    twaits = []
    h = []
    for y in ts:
        if y >= ymax:
            ymax = y
            if twait>0:
                #print(t, twait ,ymax)
                twaits.append(twait)
                twait = 0
        else:
            twait += 1
        h.append(ymax)
    max_loss = (np.array(h) - ts).max()
    twaits = np.array(twaits) if len(twaits) else np.array([len(ts)])
    return np.median(twaits), twaits.max(), max_loss, h

def convert_hist(t0, t1, btest):
    dates2load = pd.date_range(start="/".join([t0.split(".")[i] for i in [1, 2, 0]]), 
                            end="/".join([t1.split(".")[i] for i in [1, 2, 0]]), 
                            freq="D")
    source_dates = [pd.to_datetime(pos.close_date).date() for pos in btest.positions]
    target_dates = [pd.to_datetime(d).date() for d in dates2load]
    profit = btest.profits.cumsum()
    balance = [0]
    unbias=False
    for d in target_dates:
        # Select balance records with same day
        day_profs = [balance[-1]] + [b for b, sd in zip(profit, source_dates) if sd == d]
        # If there are records for currend day, store latest of them, else fill days with no records with latest sored record
        balance.append(day_profs[-1])
    res = np.array(balance)
    if unbias:
        res = res - profit[0]
    return res

In [ ]:
t0 = "2017.01.01"
t1 = "2024.01.01"
ids = list(map(int, opt_res.test_ids.iloc[opt_res_id].split(".")[1:]))
ids = [10, 31]
test1 = convert_hist(t0, t1, btests[ids[0]])
test2 = convert_hist(t0, t1, btests[ids[1]])
test_av = (test1+test2)/2
tw_mean, tw_max, loss_max, prof_max = calc_metrics(test_av)

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(test1)
plt.plot(test2)
plt.plot(test2)
plt.plot(test_av)
plt.plot(prof_max, alpha=0.4)
print(test_av[-1], tw_mean, tw_max, loss_max/test_av[-1]*100)